# Random Gallery

In [1]:
# imports
import numpy as np

import datetime

from astropy.coordinates import SkyCoord
from astropy import units

# Local
import results

# Load

In [2]:
evals_tbl = results.load_log_prob('std')

Loading: /home/xavier/Projects/Oceanography/AI/OOD/Evaluations/R2010_on_2011_95clear_128x128_preproc_std_log_prob.csv
Loading: /home/xavier/Projects/Oceanography/AI/OOD/Evaluations/R2010_on_2008_95clear_128x128_preproc_std_log_prob.csv
Loading: /home/xavier/Projects/Oceanography/AI/OOD/Evaluations/R2010_on_2009_95clear_128x128_preproc_std_log_prob.csv
Loading: /home/xavier/Projects/Oceanography/AI/OOD/Evaluations/R2010_on_2012_95clear_128x128_preproc_std_log_prob.csv
NEED TO ADD IN 2010!!!


# Cut to Top 1000

In [3]:
isrt = np.argsort(evals_tbl.log_likelihood)

In [4]:
top1000 = evals_tbl.iloc[isrt[0:1000]]
top1000.head()

,filename,row,column,latitude,longitude,clear_fraction,mean_temperature,Tmin,Tmax,T90,T10,log_likelihood,date
517380,AQUA_MODIS.20120807T233508.L2.SST.nc,189,298,-40.403057,18.733803,0.049866,12.348337,9.900000,19.154999,13.770225,11.410000,-5374.4717,2012-08-07 23:35:00
193211,AQUA_MODIS.20120807T233508.L2.SST.nc,199,295,-40.480717,18.654596,0.049988,12.404154,9.900000,19.205000,14.535967,11.384999,-5048.6953,2012-08-07 23:35:00
676391,AQUA_MODIS.20080303T065506.L2.SST.nc,1118,664,34.994330,-74.410810,0.049866,22.194897,9.915000,25.109999,24.545000,20.170000,-4193.4340,2008-03-03 06:55:00
187070,AQUA_MODIS.20090502T234508.L2.SST.nc,891,229,-42.156464,15.301422,0.049927,17.429016,NaN,NaN,NaN,NaN,-4056.6597,2009-05-02 23:45:00
721916,AQUA_MODIS.20080303T065506.L2.SST.nc,1151,645,34.733830,-74.699670,0.048950,22.161898,9.309999,25.150000,24.584999,20.150000,-4009.9785,2008-03-03 06:55:00


## Coords

In [5]:
coords = SkyCoord(b=top1000.latitude*units.deg, l=top1000.longitude*units.deg, frame='galactic')

# Loop on year

In [7]:
dyear = 1
used_coords = None
for_gallery = []
for year in [2008, 2009, 2011, 2012]:
    # Cut on date
    t0 = datetime.datetime(year,1,1)
    t1 = datetime.datetime(year+dyear,1,1)
    in_time = np.where((top1000.date >= t0) & (top1000.date < t1))[0]
    print('Year {}:, n_options={}'.format(year, len(in_time)))
    # Grab one
    if used_coords is not None:
        # Ugly loop
        max_sep = 0*units.deg
        all_seps = np.zeros((len(used_coords), len(in_time)))
        for kk, ucoord in enumerate(used_coords):
            seps = ucoord.separation(coords[in_time])
            all_seps[kk, :] = seps.to('deg').value
        # Minimum for each
        min_seps = np.min(all_seps, axis=0)
        best = np.argmax(min_seps)
        print(np.max(min_seps))
        for_gallery.append(in_time[best])
        used_coords = coords[np.array(for_gallery)]
    else:
        # Take a random one
        rani = np.random.randint(low=0,high=len(in_time),size=1)[0]
        for_gallery.append(in_time[rani])
        used_coords = coords[np.array(for_gallery)]

Year 2008:, n_options=214
Year 2009:, n_options=261
171.72235886543098
Year 2011:, n_options=302
90.39403335366364
Year 2012:, n_options=223
74.20975121981954


In [8]:
for_gallery

[692, 367, 428, 430]

In [9]:
top1000.iloc[for_gallery]

,filename,row,column,latitude,longitude,clear_fraction,mean_temperature,Tmin,Tmax,T90,T10,log_likelihood,date
513131,AQUA_MODIS.20080219T043007.L2.SST.nc,1504,780,-48.011192,-51.545025,0.049927,13.312721,8.425000,16.029999,15.580000,9.587150,-1536.7554,2008-02-19 04:30:00
362774,AQUA_MODIS.20090205T172508.L2.SST.nc,73,709,40.058464,131.659240,0.049927,6.172578,NaN,NaN,NaN,NaN,-1778.5741,2009-02-05 17:25:00
287171,AQUA_MODIS.20111107T065507.L2.SST.nc,272,997,41.254044,-67.634810,0.032776,13.444683,10.325000,23.320000,14.565000,11.905000,-1736.0500,2011-11-07 06:55:00
377267,AQUA_MODIS.20120727T220009.L2.SST.nc,216,231,12.943376,51.537080,0.049622,27.868130,22.734999,31.599998,30.914999,23.154999,-1734.2334,2012-07-27 22:00:00
